In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
food_data = pd.read_csv('서울관광재단_식당품질정보_20230111.csv', encoding = 'cp949')

food_data.head()

,식당(ID),식당명,지점명,지역명,어워드정보설명,(RTI)지수,온라인화진행여부,수용태세지수,인기도,트립어드바이저평점,씨트립평점,네이버평점
0,10012,명동정,NaN,중구,모범음식점(2015),4.213839,N,0.370,0.33,NaN,NaN,4.40
1,10014,봉추찜닭,명동중앙우체국점,중구,모범음식점(2019),4.423649,Y,0.330,0.51,NaN,4.7,4.45
2,10015,금계찜닭,명동점,중구,NaN,NaN,Y,0.082,NaN,NaN,NaN,NaN
3,10016,명동한우방,명동본점,중구,NaN,3.922220,N,0.244,0.09,NaN,NaN,4.54
4,10018,삼호복집,NaN,서대문구,모범음식점(2004),4.400069,Y,0.307,0.33,4.0,NaN,4.33


In [54]:
n_arr = pd.DataFrame({})
name_list = ['송파구','동작구','강서구','노원구','강동구']
for name in name_list:
    arr = food_data[food_data['지역명'] == name]
    n_arr = pd.concat([n_arr, arr], axis = 0)

key_word = n_arr['지역명'] + ' ' + n_arr['식당명']
n_arr.reset_index(inplace = True, drop = True)

In [55]:
type(key_word)

pandas.core.series.Series

In [56]:
len(key_word)

27951

In [57]:
n_arr['사이트'] = np.nan
n_arr['도로주소'] = np.nan
n_arr['카테고리'] = np.nan
n_arr['경도'] = np.nan
n_arr['위도'] = np.nan

url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='
headers = {
    'Authorization': 'KakaoAK 9cf3dd9afbeceb8aa9d222dd273eb4f0'
}
for i in n_arr.index:
    places = requests.get(url + key_word.iloc[i], headers=headers).json()['documents']
    if places:
        places = places[0]
        if places.get('place_url'):
            n_arr.loc[i, '사이트'] = places['place_url']
      
        n_arr.loc[i, '도로주소'] = places['road_address_name']
        n_arr.loc[i, '카테고리'] = places['category_name']
        n_arr.loc[i, '경도'] = places['x']
        n_arr.loc[i, '위도'] = places['y']


KeyboardInterrupt: 

In [58]:
print(i)

27338


In [59]:
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='
headers = {
    'Authorization': 'KakaoAK 9cf3dd9afbeceb8aa9d222dd273eb4f0'
}
for i in range(27338, 27950):
    places = requests.get(url + key_word.iloc[i], headers=headers).json()['documents']
    if places:
        places = places[0]
        if places.get('place_url'):
            n_arr.loc[i, '사이트'] = places['place_url']
      
        n_arr.loc[i, '도로주소'] = places['road_address_name']
        n_arr.loc[i, '카테고리'] = places['category_name']
        n_arr.loc[i, '경도'] = places['x']
        n_arr.loc[i, '위도'] = places['y']
    

In [62]:
n_arr['카테고리'].isnull().value_counts()

False    22194
True      5757
Name: 카테고리, dtype: int64

In [63]:
n_arr.to_excel('final_data.xlsx')